### Import Data

In [1]:
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
f = open('../Data/batsmen.json',) 
batting = json.load(f)   

In [3]:
f = open('../Data/bowlers.json',) 
bowling = json.load(f)  

In [4]:
matches = pd.read_csv("../Data/matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [5]:
f = open('../Data/scorecard.json',) 
scorecard = json.load(f)

### Final Data : Batting

In [6]:
def allBatsmen(batting):
    batting_keys=batting.keys()
    performance={}
    for k in batting_keys:
        for j in batting[k]:
            performance.setdefault(k,[]).append([j[0],j[1][1],j[1][2],j[1][3],j[1][4],j[1][5]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [7]:
bat={}
bat=allBatsmen(batting)
print(len(bat))

2099


### Final Data : Bowling

In [8]:
def allBowlers(bowling):
    bowling_keys=bowling.keys()
    performance={}
    for k in bowling_keys:
        for j in bowling[k]:
            performance.setdefault(k,[]).append([j[0],j[1][0],j[1][1],j[1][2],j[1][3],j[1][4]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [9]:
bowl={}
bowl=allBowlers(bowling)
print(len(bowl))

1616


### Batting Data for each team

In [10]:
def getLatestPerf_Bat(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [11]:
def getAvgBat(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
            
        para1=[x if x!=-1 else np.nan for x in para]
        para1=pd.Series(para1)
        f1=para1.ewm(com=0.5).mean()
        final_score.append(round(f1.mean(),3))
    return final_score

### Bowling Data for each team

In [12]:
def getLatestPerf_Bowl(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [13]:
def getAvgBowl(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
            
        para1=[x if x!=-1 else np.nan for x in para]
        para1=pd.Series(para1)
        f1=para1.ewm(com=0.5).mean()
        final_score.append(round(f1.mean(),3))
    return final_score

### Example Input

In [ ]:
#Example Input

In [14]:
team_1 = 'IND'
team_2 = 'ZIM' 
date = '2015-07-12'

In [15]:
matches_input=matches[(matches['Date']==date)]
matches_input=matches_input[(matches['Team_1']==team_1)]
match_code=matches_input['MatchCode'].iloc[0]

<ipython-input-15-dcb46b164f1d>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_input=matches_input[(matches['Team_1']==team_1)]


In [16]:
team_1_batsmen = [scorecard[str(match_code)]['BATTING1'][i][0] for i in range(10)]
team_2_batsmen = [scorecard[str(match_code)]['BATTING2'][i][0] for i in range(10)]

In [17]:
team_1_bowlers = [scorecard[str(match_code)]['BOWLING1'][i][0] for i in range(len(scorecard[str(match_code)]['BOWLING1']))]
team_2_bowlers = [scorecard[str(match_code)]['BOWLING2'][i][0] for i in range(len(scorecard[str(match_code)]['BOWLING2']))]

In [18]:
team1_bat=getLatestPerf_Bat(bat,team_1_batsmen, date, 5)
team2_bat=getLatestPerf_Bat(bat,team_2_batsmen, date, 5)

In [19]:
team1_bowl=getLatestPerf_Bowl(bowl,team_1_bowlers, date, 5)
team2_bowl=getLatestPerf_Bowl(bowl,team_2_bowlers, date, 5)

In [20]:
data = [[date,team_1, team_2]] 

predict = pd.DataFrame(data, columns = ['Date','Team1', 'Team2'])

In [23]:
predict

,Date,Team1,Team2
0,2015-07-12,IND,ZIM


In [24]:
predict['Runs_team1'] = ''
predict['Runs_team2'] = ''

predict['Fours_team1'] = ''
predict['Fours_team2'] = ''
predict['Sixes_team1'] = ''
predict['Sixes_team2'] = ''
predict['Strike_rate_team1'] = ''
predict['Strike_rate_team2'] = ''
predict['Maidens_team1'] = ''
predict['Maidens_team2'] = ''
predict['Wickets_team1'] = ''
predict['Wickets_team2'] = ''
predict['Economy_rate_team1'] = ''
predict['Economy_rate_team2'] = ''

In [25]:
predict['Runs_team1'].iloc[0] = getAvgBat(team1_bat,1)
predict['Runs_team2'].iloc[0] = getAvgBat(team2_bat,1)
predict['Fours_team1'].iloc[0] = getAvgBat(team1_bat,3)
predict['Fours_team2'].iloc[0] = getAvgBat(team2_bat,3)
predict['Sixes_team1'].iloc[0] = getAvgBat(team1_bat,4)
predict['Sixes_team2'].iloc[0] = getAvgBat(team2_bat,4)
predict['Strike_rate_team1'].iloc[0] = getAvgBat(team1_bat,5)
predict['Strike_rate_team2'].iloc[0] = getAvgBat(team2_bat,5)

In [30]:
def adjust_length(l):
    if (len(l)<10):
        l.extend([-1] * (10 - len(l)))
    return l

In [32]:
predict['Maidens_team1'].iloc[0] = adjust_length(getAvgBowl(team1_bowl,1))
predict['Maidens_team2'].iloc[0] = adjust_length(getAvgBowl(team2_bowl,1))
predict['Wickets_team1'].iloc[0] = adjust_length(getAvgBowl(team1_bowl,3))
predict['Wickets_team2'].iloc[0] = adjust_length(getAvgBowl(team2_bowl,3))
predict['Economy_rate_team1'].iloc[0] = adjust_length(getAvgBowl(team1_bowl,4))
predict['Economy_rate_team2'].iloc[0] = adjust_length(getAvgBowl(team2_bowl,4))

In [33]:
predict

,Date,Team1,Team2,Runs_team1,Runs_team2,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2
0,2015-07-12,IND,ZIM,"[30.115, 16.138, 41.229, 31.892, 9.904, 25.32,...","[27.694, 23.366, 28.702, 24.965, 28.928, 29.36...","[3.03, 1.697, 3.39, 3.06, 0.885, 2.269, 1.75, ...","[2.882, 2.581, 2.743, 2.199, 2.378, 3.183, 1.6...","[0.345, 0.072, 0.586, 0.126, 0.0, 0.639, 0.0, ...","[0.429, 0.286, 0.493, 0.456, 0.471, 0.618, 0.1...","[62.179, 53.124, 77.292, 73.897, 52.657, 73.92...","[60.614, 58.087, 61.916, 73.845, 74.74, 73.004...","[8.462, 7.803, 7.557, 6.49, 7.714, 3.268, 3.30...","[8.406, 8.618, 9.484, 6.914, 9.067, -1, -1, -1...","[50.827, 46.914, 49.337, 32.396, 38.1, 15.659,...","[39.109, 47.481, 40.596, 36.524, 39.371, -1, -...","[1.284, 2.236, 1.609, 0.589, 0.824, 0.318, 0.3...","[0.993, 1.895, 1.255, 1.353, 1.622, -1, -1, -1..."
